In [0]:
# Após fazer o upload da base de dados no FileStore
file_location = "/FileStore/tables/ecommerce_dataset_updated.csv"
file_type = "csv"
from pyspark.sql.functions import sum, col, when, count, round, to_date, col, month, year
from pyspark.sql import functions as F # só pelo order by
from pyspark.sql.types import DoubleType
import plotly.express as px
import plotly.graph_objects as go


infer_schema = "false"
first_row_is_header = "false"
delimiter = ","

df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", "true") \
  .option("sep", delimiter) \
  .load(file_location)

# Renomeando alalgumas colunas
df = df.withColumnRenamed("Final_Price(Rs.)", "Final_Price") \
       .withColumnRenamed("Discount (%)", "Discount") \
       .withColumnRenamed("Price (Rs.)", "Price")

# Alterando a coluna data de string pata date.
df = df.withColumn("Purchase_Date", to_date(col("Purchase_Date"), "dd-MM-yyyy"))
display(df.limit(10))

User_ID,Product_ID,Category,Price,Discount,Final_Price,Payment_Method,Purchase_Date
337c166f,f414122f-e,Sports,36.53,15,31.05,Net Banking,2024-11-12
d38a19bf,fde50f9c-5,Clothing,232.79,20,186.23,Net Banking,2024-02-09
d7f5f0b0,0d96fc90-3,Sports,317.02,25,237.76,Credit Card,2024-09-01
395d4994,964fc44b-d,Toys,173.19,25,129.89,UPI,2024-04-01
a83c145c,d70e2fc6-e,Beauty,244.8,20,195.84,Net Banking,2024-09-27
3fdcdae8,0816ee12-5,Books,241.86,50,120.93,UPI,2024-08-08
12772337,d41e0b55-c,Toys,76.91,5,73.06,Credit Card,2024-03-28
9f0194af,83b7dc04-5,Sports,213.23,20,170.58,Net Banking,2024-05-29
7a8559d8,87c9c896-1,Sports,379.81,5,360.82,Credit Card,2024-02-26
8911f0d1,31308875-b,Home & Kitchen,415.36,50,207.68,Net Banking,2024-01-11


#### Vendas por mês

In [0]:
# Criando as colunas month e year a partir da coluna Purchase_Date
mes_ano = df.withColumn("Month", month("Purchase_Date")) \
                     .withColumn("Year", year("Purchase_Date"))

# Agrupar por ano e mês e somar as vendas
sales_mes = mes_ano.groupBy("Year", "Month").agg(round(sum("Final_Price"), 2).alias("Total_Vendas"))

# Ordenar pelo mês (ano, mês)
sales_mes = sales_mes.orderBy(["Year", "Month"], ascending=[True, True])

# Exibir o resultado
display(sales_mes)

Year,Month,Total_Vendas
2024,1,69009.43
2024,2,69186.34
2024,3,71325.4
2024,4,74365.13
2024,5,68837.25
2024,6,64256.66
2024,7,72798.23
2024,8,71506.96
2024,9,68043.09
2024,10,76034.51


##### Gráfico de vendas por mês

In [0]:
# Utilizando o Plotly para fazermos nosso primeiro gráfico com a biblioteca
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=sales_mes.toPandas()['Month'],
    y=sales_mes.toPandas()['Total_Vendas'],
    mode='lines+markers',  
    marker=dict(size=8, color='red', symbol='circle'), 
    line=dict(color='cyan', width=2),  
))

# Personalizar o layout
fig.update_layout(
    title='Total de Vendas por Mês',
    xaxis_title='Mês',
    yaxis_title='Total de Vendas',
    xaxis=dict(tickmode='array', tickvals=sales_mes.toPandas()['Month']),
    plot_bgcolor='black',  # Cor do fundo do gráfico
    paper_bgcolor='black',  # Cor do fundo da área do gráfico
    font=dict(color='white'),  # Cor da fonte para branco
    showlegend=False  # Remover legenda
)

fig.show()

#### Vendas por categoria

In [0]:
df_vendas_categoria = df.groupBy("Category").agg(round(sum("Final_Price"), 2).alias("Vendas por Categoria"))
df_vendas_categoria = df_vendas_categoria.orderBy(F.col("Vendas por Categoria"), ascending=False)
display(df_vendas_categoria)

Category,Vendas por Categoria
Clothing,115314.84
Books,111149.35
Home & Kitchen,110328.08
Sports,108518.79
Toys,107289.69
Beauty,104215.1
Electronics,100462.23


##### Gráfico de vendas por categoria

In [0]:
df_pandas = df_vendas_categoria.toPandas()
fig = px.bar(df_pandas, x="Category", y="Vendas por Categoria", title="Vendas por Categoria", color="Category")
fig.show()

#### Desconto por categoria

In [0]:
# Converter a coluna de desconto para double para calcular em seguida o desconto
df = df.withColumn("Discount", col("Discount").cast("double"))

# Substituir valores NaN (null) por 0 caso tenha
df = df.fillna({"Discount": 0})

# Calcular o valor do desconto para cada item
df = df.withColumn("Discount_Amount", col("Price") - col("Final_Price"))

# Agrupar por categoria e calcular o total de desconto
desconto_por_categoria = df.groupBy("Category").agg(round(sum("Discount_Amount"), 2).alias("Desconto por Categoria"))

# Ordenar do maior para o menor desconto
desconto_por_categoria = desconto_por_categoria.orderBy(col("Desconto por Categoria").desc())
display(desconto_por_categoria)

Category,Desconto por Categoria
Books,27222.36
Home & Kitchen,26513.43
Sports,25963.2
Electronics,24857.91
Clothing,24399.71
Beauty,23417.94
Toys,22917.84


##### Gráfico desconto por categoria

In [0]:
# Convertendo para o Pandas
df_descontos = desconto_por_categoria.toPandas()

fig = px.bar(df_descontos, x="Category", y="Desconto por Categoria", title="Total de Descontos por Categoria", text_auto=True)
fig.show()

#### Total método de pagamento

In [0]:
# Contando a frequência de cada método de pagamento
df_metodos = df.groupBy("Payment_Method").agg(count("*").alias("Total Método de Pagamento"))

# Ordenar do maior para o menor
df_metodos = df_metodos.orderBy(col("Total Método de Pagamento").desc())

display(df_metodos)

Payment_Method,Total Método de Pagamento
Credit Card,760
UPI,757
Debit Card,731
Net Banking,716
Cash on Delivery,696


##### Gráfico total método de pagamento

In [0]:
df_pagamento = df_metodos.toPandas()

# Criar gráfico de pizza
fig = px.pie(df_pagamento, names="Payment_Method", values="Total Método de Pagamento", title="Distribuição de Formas de Pagamento")
fig.show()